In [38]:
from docplex.mp.model import Model
import numpy as np
from sklearn.linear_model import LinearRegression
import docplex.mp.conflict_refiner as cr

In [37]:
def galo_bids_init(value_model, bidder_id, n, presampled_n, presampled_algorithm, ml_model):
    np.random.seed(0)
    values = np.random.randint(0, 100, size=(10)).tolist()
    bundles = np.random.randint(0, 2, size=(10, 5)).tolist()
    #print(bundles)
    M = len(bundles[0])

    # bundles random 18 bundles of length 10
    reg = LinearRegression().fit(bundles, values)

    count = 10
    bigM = 300
    while count <= 30:
        reg = LinearRegression().fit(bundles, values)
        coef = reg.coef_
        intercept = reg.intercept_
        print(coef)
        print(intercept)
        x_vectors = []
        x_distances = []
        for bundle in range(len(bundles)):

            m = Model(name='galo')

            r = m.continuous_var(name='r')
            
            x = m.binary_var_list(range(M), name='x')
            
            b = [m.binary_var(name=f'b_{j}') for j in range(len(values))]

            constraints = []

            y_prediction = m.sum(x[k]*coef[k] for k in range(M)) + intercept


            constraints.append(values[bundle] - y_prediction <= r)
            constraints.append(y_prediction - values[bundle] <= r)

            for j in range(len(values)):
                constraints.append(values[j] - y_prediction + bigM * b[j] >= r)
                constraints.append(y_prediction - values[j] + bigM * (1 - b[j]) >= r)

            m.add_constraints(constraints)
            
            m.maximize(r)
            
            sol = m.solve()

            res = m.get_solve_status()

            if res.name == 'INFEASIBLE_SOLUTION':  # or also 'INFEASIBLE_OR_UNBOUNDED_SOLUTION'
                cref = cr.ConflictRefiner()
                print('show some of the constraints that can be removed to arrive at a minimal conflict')
                confl = cref.refine_conflict(m, display=True)  # display flag is to show the conflicts
                print('Initial y')
                print(values[bundle])
                print('Intercept')
                print(intercept)
                cref.display_conflicts(confl)

            try:
                print(sol.solve_status)

            except:
                print('No solution found')
                continue

            vector = np.array([x[k].solution_value for k in range(M)])
            x_vectors.append(vector)
            distance = np.sum(np.abs(np.sum(vector * coef) + intercept - values[bundle]))
            x_distances.append(distance)

        chosen_bundle = x_vectors[np.argmax(x_distances)]
        #generate one random value
        value = np.random.randint(0, 100)
        values.append(value)
        bundles.append(chosen_bundle.tolist())
        count+=1

    return bundles

results = galo_bids_init(None, 0, 20, 10, 'unif', None)
#results

[  6.88679245  38.73584906  -2.88679245   3.8490566  -18.86792453]
35.58490566037738
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION


JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
[  3.796  25.1     0.204  22.212 -26.504]
30.676000000000016
No solution found
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
show some of the constraints that can be removed to arrive at a minimal conflict
Initial y
67
Intercept
30.676000000000016
Conflict set:
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -3.796x_0-25.100x_1-0.204x_2-22.212x_3+26.504x_4+36.324 <= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 3.796x_0+25.100x_1+0.204x_2+22.212x_3-26.504x_4-36.324 <= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -3.796x_0-25.100x_1-0.204x_2-22.212x_3+26.504x_4+300b_2+33.324 >= r
	<class 'docplex.mp.con

/home/artem/.conda/envs/ica_new/lib/python3.6/site-packages/docplex/mp/model.py:3278: UserWarning: Unknown quick-setting in Context: display, value: True
  context.update_key_value(argname, argval)


show some of the constraints that can be removed to arrive at a minimal conflict


/home/artem/.conda/envs/ica_new/lib/python3.6/site-packages/docplex/mp/model.py:3278: UserWarning: Unknown quick-setting in Context: display, value: True
  context.update_key_value(argname, argval)


...

/home/artem/.conda/envs/ica_new/lib/python3.6/site-packages/docplex/mp/model.py:3278: UserWarning: Unknown quick-setting in Context: display, value: True
  context.update_key_value(argname, argval)


Initial y
10
Intercept
23.149444600018654
Conflict set:
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.431x_0-18.981x_1+0.587x_2-19.386x_3+17.545x_4-13.149 <= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 0.431x_0+18.981x_1-0.587x_2+19.386x_3-17.545x_4+13.149 <= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.431x_0-18.981x_1+0.587x_2-19.386x_3+17.545x_4+300b_5-14.149 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 0.431x_0+18.981x_1-0.587x_2+19.386x_3-17.545x_4-300b_5+314.149 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.431x_0-18.981x_1+0.587x_2-19.386x_3+17.545x_4+300b_7-2.149 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 0.431x_0+18.981x_1-0.587x_2+19.386x_3-17.545x_4-300b_7+302.149 >= r
No solution found
show some of the constraints t

/home/artem/.conda/envs/ica_new/lib/python3.6/site-packages/docplex/mp/model.py:3278: UserWarning: Unknown quick-setting in Context: display, value: True
  context.update_key_value(argname, argval)


show some of the constraints that can be removed to arrive at a minimal conflict
Initial y
10
Intercept
23.149444600018654
Conflict set:
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.431x_0-18.981x_1+0.587x_2-19.386x_3+17.545x_4-13.149 <= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 0.431x_0+18.981x_1-0.587x_2+19.386x_3-17.545x_4+13.149 <= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.431x_0-18.981x_1+0.587x_2-19.386x_3+17.545x_4+300b_5-14.149 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 0.431x_0+18.981x_1-0.587x_2+19.386x_3-17.545x_4-300b_5+314.149 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.431x_0-18.981x_1+0.587x_2-19.386x_3+17.545x_4+300b_7-2.149 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 0.431x_0+18.981x_1-0.587x_2+19.386x

/home/artem/.conda/envs/ica_new/lib/python3.6/site-packages/docplex/mp/model.py:3278: UserWarning: Unknown quick-setting in Context: display, value: True
  context.update_key_value(argname, argval)
